## The Integer Partition Problem

In the _integer partition_ problem we seek to partition the elements of a set $ S $ in two sets $ A $ and $ B $ such that $ \sum_{a \in A} a = \sum_{b \in B} b $ or alternatively make the difference as small as possible [3].

### Model Formulation

#### Decision Variables

For each $ x_i \in S $ we create two variables $ in_{A}^i $ and $ in_{B}^i $. These variables can only take values in $ \{0,1\} $. And $ in_{P}^i = 1 $ if and only if $ x_i \in P $ for $ P \in \{A, B\} $.

#### Constraints

An element can only belong to one set in the resulting partition. So for each $ x_i \in S $

$$ in_A^i + in_B^i = 1. $$

In addition, the difference must be minimized, but remain greater than 0.

$$ \sum_{a \in A}a - \sum_{b \in B}b \geq 0 $$

#### Objective

Since a solution to the "strict" partition problem does not always exists, we will try to minimize the difference of the two partitions. When the solution exits, the difference is equal to $ 0 $ and it is the smallest it can get because of the non-negativity constraint.

$$ Minimize \sum_{a \in A}a - \sum_{b \in B}b $$


### Model Deployment

We begin by importing the necessary modules. Next we define a reusable function to run a model. The function `integer_partition` takes a list of integers defining an instance of the integer partition problem. It declares declare the decision variables then adds the constraints. The function `print_solution` just prints a solution returned by the function `integer_partition`.

In [4]:
import sys
import random
import gurobipy as gb

from collections import defaultdict
from gurobipy import GRB


def integer_partition(S, fname=None):
    """Return two lists solution to the integer partition on S.
    
    Args:
        S: iterable of integers
        fname: OPTIONAL output file for the generated model
    """
    
    N = len(S)
    
    m = gb.Model('IP_{}'.format(fname))

    # decision variables for model integer partition
    partition = m.addVars(['A', 'B'], range(N)
                          , name="in"
                          , vtype=GRB.BINARY)

    # each elements must be in exactly one set in the partition
    separation = m.addConstrs((partition['A', i] + partition['B', i] == 1 for i in range(N))
                              , name="separation")

    # Constraint to keep the difference non negative
    min_diff = m.addConstr(sum(S[i]*partition['A',i] 
                                for i in range(N))
                            -sum(S[i]*partition['B',i] 
                                 for i in range(N)) >= 0
                          , name='minimize_difference')

    m.setObjective(sum(S[i]*partition['A',i] for i in range(N))
                   - sum(S[i]*partition['B',i] for i in range(N))
                   , GRB.MINIMIZE)

    # Save generated model for inspection.
    m.write('files/{}'.format(fname or 'integer_partition.lp'))
    m.optimize()
    result = defaultdict(list)

    # add each element of nums to the set to which it belongs in the partition
    for s, i in partition:
        if partition[s, i].x >= sys.float_info.epsilon:
            result[s].append(S[i])
            
    return result


def print_solution(result):
    """Print a solution to the integer partition problem."""
    for s, items in result.items():
        print('sum({}) = sum({}) = {}'.format(s, items, sum(items)))


### Running the model

We now run the model on two instances, one for which we know it has at least two solutions and the other, a random instance.

In [5]:
S = [423, 779, 434, 371, 244, 245, 753, 519, 106, 167, 34, 650,
     865, 605, 441, 190, 774, 512, 970, 394, 518, 887, 908, 971, 14]


print_solution( integer_partition(S, 'ip_feasible.lp') )

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 26 rows, 50 columns and 100 nonzeros
Model fingerprint: 0x7190bcc4
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1434.0000000
Presolve removed 26 rows and 50 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
sum(A) = sum([779, 244, 190, 512, 970, 394, 518, 887, 908, 971, 14]) = 6387
sum(B) = sum([423, 434, 371, 245, 753, 519, 106, 167, 34, 650, 865, 605, 441, 774]) = 6387


In [7]:
# Uncomment to try with a random list
# S = list(random.choices(range(1000), k=21))

S = [372, 734, 954, 124, 985, 759, 785, 462, 522, 70, 204, 751, 343, 57, 152, 209, 724, 405, 867, 177, 701]

print_solution( integer_partition(S, 'ip_infeasible.lp') )

[372, 734, 954, 124, 985, 759, 785, 462, 522, 70, 204, 751, 343, 57, 152, 209, 724, 405, 867, 177, 701]
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 22 rows, 42 columns and 84 nonzeros
Model fingerprint: 0x39dc88a7
Variable types: 0 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [6e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3063.0000000
Presolve removed 22 rows and 42 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.18 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
sum(A) = sum([759, 785, 343, 57, 152, 209, 724, 405, 867, 177, 701]) = 5179
sum(B) = sum([372, 734, 954, 124, 985, 462, 522, 70, 204, 751]) 